In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans # import k-means from clustering stage
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [2]:
#Getting the csv file with coordinates and storing it in a dataframe
!wget --quiet https://raw.githubusercontent.com/maydinel/Coursera_Capstone/master/map%20-%20kadikoy%20-%20v5.csv
df_coord = pd.read_csv('map - kadikoy - v5.csv')
print(df_coord.shape)
df_coord.head()

(26, 6)


,lon,lat,marker-color,marker-size,marker-symbol,name
0,29.022317,40.979963,#7e7e7e,medium,NaN,A1
1,29.025385,40.981405,#7e7e7e,medium,NaN,A2
2,29.028990,40.981745,#7e7e7e,medium,NaN,A3
3,29.032145,40.982911,#7e7e7e,medium,NaN,A4
4,29.029870,40.984952,#7e7e7e,medium,NaN,B3


In [3]:
#Dropping the unnecessary columns
df_coord.drop(['marker-color', 'marker-size', 'marker-symbol'], axis=1, inplace = True)
print(df_coord.shape)
df_coord.head()

(26, 3)


,lon,lat,name
0,29.022317,40.979963,A1
1,29.025385,40.981405,A2
2,29.028990,40.981745,A3
3,29.032145,40.982911,A4
4,29.029870,40.984952,B3


In [4]:
!conda install -c conda-forge folium=0.5.0 --yes # installing folium
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [5]:
#Creating map of Kadikoy
map_kadikoy = folium.Map(location=[40.9890, 29.0275], zoom_start=15)

#Adding markers to the map
for lat, lon, name in zip(df_coord['lat'], df_coord['lon'], df_coord['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kadikoy)  
    
map_kadikoy

Entering the Credentials for the Foursquare API

In [1]:
# The code was removed by Watson Studio for sharing.

In [9]:
#Checking results for a certain data point
df_coord.loc[0, 'name']

'A1'

In [10]:
neighborhood_latitude = df_coord.loc[0, 'lat'] # neighborhood latitude value
neighborhood_longitude = df_coord.loc[0, 'lon'] # neighborhood longitude value

neighborhood_name = df_coord.loc[0, 'name'] # region code

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of A1 are 40.979962866583264, 29.02231693267822.


In [11]:
# @hidden cell

LIMIT = 10000 # limit of number of venues returned by Foursquare API

radius = 150 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EAUSZ3BTGKL3XCTW5CFM3DH1YUG5XL4A3R1SHE2FM4YR2HBA&client_secret=PMYI2KYXCVXFACD14DH0IDQR2R0GT1FV1XYRJNBKT1RJXMQX&v=20190625&ll=40.979962866583264,29.02231693267822&radius=150&limit=10000'

In [12]:
#Retrieving the results from the Foursquare Places API
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d128fa986bc49002c8c2989'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-56054cd3498e96c19f6aa480-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16d941735',
         'name': 'Café',
         'pluralName': 'Cafés',
         'primary': True,
         'shortName': 'Café'}],
       'id': '56054cd3498e96c19f6aa480',
       'location': {'address': 'Caferağa Mah. Osman Zeki Üngör Sok. No:15, Moda-Kadıköy',
        'cc': 'TR',
        'city': 'İstanbul',
        'country': 'Türkiye',
        'distance': 97,
        'formattedAddress': ['Caferağa Mah. Osman Zeki Üngör Sok. No:15, Moda-Kadıköy',
         '34710 İstanbul',
         'Türkiye'],
        'labeledLatLngs': 

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Cafe MOON,Café,40.980442,29.021340
1,Cafe Los Manços,Café,40.981140,29.022968
2,Moda Parkı,Park,40.980748,29.021341
3,Nefess İstanbul,Yoga Studio,40.980128,29.022858
4,Tatlı Mesai,Café,40.980285,29.023170


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

33 venues were returned by Foursquare.


In [16]:
#Function for getting the venues nearby
def getNearbyVenues(name, lat, lng, radius=150):
    
    venues_list=[]
    for name, lat, lng in zip(name, lat, lng):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
kadikoy_venues = getNearbyVenues(name=df_coord['name'],
                                   lat=df_coord['lat'],
                                   lng=df_coord['lon']
                                  )

A1
A2
A3
A4
B3
B2
B1
C3
D1
D2
D3
D4
D5
E5
E4
E3
E1
F2
F3
G2
G1
C4
E2
F1
C2
C1


In [23]:
print(kadikoy_venues.shape)
kadikoy_venues.head()

(687, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,A1,40.979963,29.022317,Cafe MOON,40.980442,29.021340,Café
1,A1,40.979963,29.022317,Cafe Los Manços,40.981140,29.022968,Café
2,A1,40.979963,29.022317,Moda Parkı,40.980748,29.021341,Park
3,A1,40.979963,29.022317,Nefess İstanbul,40.980128,29.022858,Yoga Studio
4,A1,40.979963,29.022317,Tatlı Mesai,40.980285,29.023170,Café


In [27]:
kadikoy_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A1,33,33,33,33,33,33
A2,15,15,15,15,15,15
A3,1,1,1,1,1,1
A4,9,9,9,9,9,9
B1,21,21,21,21,21,21
B2,71,71,71,71,71,71
B3,2,2,2,2,2,2
C1,16,16,16,16,16,16
C2,77,77,77,77,77,77


In [28]:
print('There are {} unique categories.'.format(len(kadikoy_venues['Venue Category'].unique())))

There are 143 unique categories.


In [29]:
# one hot encoding
kadikoy_onehot = pd.get_dummies(kadikoy_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kadikoy_onehot['Neighborhood'] = kadikoy_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kadikoy_onehot.columns[-1]] + list(kadikoy_onehot.columns[:-1])
kadikoy_onehot = kadikoy_onehot[fixed_columns]

kadikoy_onehot.head()

,Neighborhood,Accessories Store,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Toy / Game Store,Trail,Turkish Home Cooking Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar,Winery,Women's Store,Yoga Studio
0,A1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,A1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
kadikoy_onehot.shape

(687, 144)

In [31]:
kadikoy_grouped = kadikoy_onehot.groupby('Neighborhood').mean().reset_index()
kadikoy_grouped

,Neighborhood,Accessories Store,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Toy / Game Store,Trail,Turkish Home Cooking Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar,Winery,Women's Store,Yoga Studio
0,A1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060606
1,A2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,A3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,A4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,B1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095238,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,B2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.014085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,B3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,C1,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.062500,0.000000,0.000000,...,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,C2,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000,0.000000,0.000000,0.012987,...,0.012987,0.0000,0.000000,0.000000,0.000000,0.000000,0.012987,0.012987,0.000000,0.000000
9,C3,0.000000,0.000000,0.153846,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000


In [32]:
kadikoy_grouped.shape

(26, 144)

In [33]:
num_top_venues = 5

for hood in kadikoy_grouped['Neighborhood']:
    print("----"+str(hood)+"----")
    temp = kadikoy_grouped[kadikoy_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A1----
           venue  freq
0           Café  0.18
1    Coffee Shop  0.15
2            Pub  0.09
3    Yoga Studio  0.06
4  Hot Dog Joint  0.06


----A2----
         venue  freq
0         Café  0.13
1  Music Venue  0.07
2     Boutique  0.07
3  Coffee Shop  0.07
4  Flower Shop  0.07


----A3----
               venue  freq
0         Restaurant   1.0
1  Accessories Store   0.0
2          Pet Store   0.0
3          Nightclub   0.0
4        Opera House   0.0


----A4----
                     venue  freq
0                     Café  0.11
1                    Hotel  0.11
2  Health & Beauty Service  0.11
3                     Pool  0.11
4               Restaurant  0.11


----B1----
            venue  freq
0            Café  0.24
1     Coffee Shop  0.10
2          Bakery  0.10
3    Concert Hall  0.05
4  Chocolate Shop  0.05


----B2----
            venue  freq
0            Café  0.28
1     Coffee Shop  0.11
2    Dessert Shop  0.06
3  Chocolate Shop  0.04
4      Restaurant  0.04


----B3----

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = kadikoy_grouped['Neighborhood']

for ind in np.arange(kadikoy_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(kadikoy_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A1,Café,Coffee Shop,Pub,Yoga Studio,Tea Room,Diner,Hot Dog Joint,Mediterranean Restaurant,Gym,Fast Food Restaurant
1,A2,Café,Music Venue,Seafood Restaurant,Boutique,Park,Turkish Restaurant,Museum,Hookah Bar,Flower Shop,Thrift / Vintage Store
2,A3,Restaurant,Yoga Studio,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Falafel Restaurant
3,A4,Pool,Café,Hotel,Athletics & Sports,Restaurant,Fast Food Restaurant,Health & Beauty Service,Park,Tennis Court,Falafel Restaurant
4,B1,Café,Coffee Shop,Bakery,Dessert Shop,Chocolate Shop,Sausage Shop,Steakhouse,Food & Drink Shop,Music Venue,Sushi Restaurant


In [36]:
# set number of clusters
kclusters = 5

kadikoy_grouped_clustering = kadikoy_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kadikoy_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 4, 2, 2, 2, 3, 2, 2, 2], dtype=int32)

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

kadikoy_merged = df_coord

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kadikoy_merged = kadikoy_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='name')

kadikoy_merged.head() # check the last columns!

,lon,lat,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,29.022317,40.979963,A1,2,Café,Coffee Shop,Pub,Yoga Studio,Tea Room,Diner,Hot Dog Joint,Mediterranean Restaurant,Gym,Fast Food Restaurant
1,29.025385,40.981405,A2,2,Café,Music Venue,Seafood Restaurant,Boutique,Park,Turkish Restaurant,Museum,Hookah Bar,Flower Shop,Thrift / Vintage Store
2,29.028990,40.981745,A3,4,Restaurant,Yoga Studio,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Falafel Restaurant
3,29.032145,40.982911,A4,2,Pool,Café,Hotel,Athletics & Sports,Restaurant,Fast Food Restaurant,Health & Beauty Service,Park,Tennis Court,Falafel Restaurant
4,29.029870,40.984952,B3,3,Music Venue,Tattoo Parlor,Yoga Studio,Gastropub,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Fast Food Restaurant


In [38]:
# create map
map_clusters = folium.Map(location=[40.9890, 29.0275], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kadikoy_merged['lat'], kadikoy_merged['lon'], kadikoy_merged['name'], kadikoy_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Observations about the Clusters:

In [39]:
kadikoy_merged.loc[kadikoy_merged['Cluster Labels'] == 0, kadikoy_merged.columns[[2] + list(range(3, kadikoy_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,E1,0,Café,Plaza,Seafood Restaurant,City Hall,Hot Dog Joint,Tea Room,Yoga Studio,Fast Food Restaurant,Food Court,Food & Drink Shop
17,F2,0,Café,Breakfast Spot,Coffee Shop,Restaurant,Art Gallery,Pub,Seafood Restaurant,Food & Drink Shop,Bistro,Dessert Shop
19,G2,0,Café,Vegetarian / Vegan Restaurant,Dance Studio,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop,Flower Shop


In [40]:
kadikoy_merged.loc[kadikoy_merged['Cluster Labels'] == 1, kadikoy_merged.columns[[2] + list(range(3, kadikoy_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,E2,1,Coffee Shop,Electronics Store,Burger Joint,Mac & Cheese Joint,Doner Restaurant,Pastry Shop,Pizza Place,Sandwich Place,Clothing Store,Candy Store
23,F1,1,Bed & Breakfast,Medical Center,Café,Burger Joint,Sandwich Place,Steakhouse,Breakfast Spot,Food Truck,Bookstore,Coffee Shop


In [41]:
kadikoy_merged.loc[kadikoy_merged['Cluster Labels'] == 2, kadikoy_merged.columns[[2] + list(range(3, kadikoy_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A1,2,Café,Coffee Shop,Pub,Yoga Studio,Tea Room,Diner,Hot Dog Joint,Mediterranean Restaurant,Gym,Fast Food Restaurant
1,A2,2,Café,Music Venue,Seafood Restaurant,Boutique,Park,Turkish Restaurant,Museum,Hookah Bar,Flower Shop,Thrift / Vintage Store
3,A4,2,Pool,Café,Hotel,Athletics & Sports,Restaurant,Fast Food Restaurant,Health & Beauty Service,Park,Tennis Court,Falafel Restaurant
5,B2,2,Café,Coffee Shop,Dessert Shop,Chocolate Shop,Restaurant,Breakfast Spot,Italian Restaurant,Comfort Food Restaurant,Bookstore,Gym / Fitness Center
6,B1,2,Café,Coffee Shop,Bakery,Dessert Shop,Chocolate Shop,Sausage Shop,Steakhouse,Food & Drink Shop,Music Venue,Sushi Restaurant
7,C3,2,Café,Arcade,Theater,Restaurant,Coffee Shop,Korean Restaurant,Tea Room,Opera House,Movie Theater,Furniture / Home Store
8,D1,2,Hotel,Café,Hotel Bar,Restaurant,Bistro,Spa,Park,Yoga Studio,Flower Shop,Food Truck
9,D2,2,Bar,Coffee Shop,Pub,Café,Bookstore,Comfort Food Restaurant,Meyhane,Kebab Restaurant,Turkish Restaurant,Music Venue
10,D3,2,Café,Pub,Bar,Meyhane,Coffee Shop,Music Venue,Pide Place,Theater,Bistro,Tattoo Parlor
11,D4,2,Café,Pub,Coffee Shop,Performing Arts Venue,Breakfast Spot,Pizza Place,Bar,Comfort Food Restaurant,Turkish Restaurant,Clothing Store


In [42]:
kadikoy_merged.loc[kadikoy_merged['Cluster Labels'] == 3, kadikoy_merged.columns[[2] + list(range(3, kadikoy_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,B3,3,Music Venue,Tattoo Parlor,Yoga Studio,Gastropub,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Fast Food Restaurant


In [43]:
kadikoy_merged.loc[kadikoy_merged['Cluster Labels'] == 4, kadikoy_merged.columns[[2] + list(range(3, kadikoy_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,A3,4,Restaurant,Yoga Studio,Fast Food Restaurant,Furniture / Home Store,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop,Flower Shop,Falafel Restaurant
